In [ ]:
import os
import re
import io
import sys
import glob
import enum
import json
import dask
import xlrd
import base64
import imageio
import requests
import datetime
import numpy as np
import pandas as pd
import skimage

import dask.diagnostics
import sqlalchemy as db
import sqlalchemy.orm
import sqlalchemy.ext.declarative
from matplotlib import pyplot as plt

%load_ext autoreload
%autoreload 1

sys.path.append('../..')
%aimport opencell.imaging.managers
%aimport opencell.imaging.processors
%aimport opencell.file_utils
# %aimport opencell.database.operations

from opencell import constants, file_utils
from opencell.cli import database as db_cli
from opencell.cli import imaging as imaging_cli
from opencell.database import models
from opencell.database import operations as ops
from opencell.database import utils as db_utils
from opencell.imaging import utils as im_utils
from opencell.imaging import images, managers, processors, viz

In [ ]:
url = db_utils.url_from_credentials('../../db-credentials-dev.json')
# url = db_utils.url_from_credentials('../db-credentials-cap.json')
url

In [ ]:
engine = db.create_engine(url)
session_factory = db.orm.sessionmaker(bind=engine)
Session = db.orm.scoped_session(session_factory)
session = Session()

### Top-scoring FOVs

In [ ]:
# throws ValueError and TypeError, respectively
d.iloc[np.nan], d.iloc[[np.nan]]

### Generate FOV thumbnails

In [ ]:
im_405 = imageio.imread(
    '/Users/keith.cheveralls/image-data/raw-pipeline-microscopy/PML0227/QC/z-projections/MMStack_115-C2-7_C0-PROJ-Z.tif')
im_488 = imageio.imread(
    '/Users/keith.cheveralls/image-data/raw-pipeline-microscopy/PML0227/QC/z-projections/MMStack_115-C2-7_C1-PROJ-Z.tif')


im_405 = im_405[:600, :600]
im_488 = im_488[:600, :600]

im_rgb = processors.FOVProcessor.make_rgb(im_405, im_488)

In [ ]:
# compare downsampled images using rescale and downscale_local_mean
viz.imshow(
    np.concatenate(
        (
            skimage.transform.rescale(im_rgb, scale=.2, order=0, anti_aliasing=False, multichannel=True),
            skimage.transform.rescale(im_rgb, scale=.2, order=1, anti_aliasing=False, multichannel=True),
            skimage.transform.rescale(im_rgb, scale=.2, order=1, anti_aliasing=True, multichannel=True),
            im_utils.autoscale(skimage.transform.downscale_local_mean(im_rgb, factors=(5, 5, 1)))[:, :, :].astype('float64')/255,
        ), 
        axis=1),
    figsize=7)

In [ ]:
fovs = session.query(models.MicroscopyFOV).all()
p = processors.FOVProcessor.from_database(fovs[0])
len(fovs), p.target_name

In [ ]:
fov = fovs[0]
th = fov.thumbnails[0]

In [ ]:
result = p.generate_fov_thumbnails('/Volumes/ml_group/opencell-microscopy/', scale=10, quality=90)

In [ ]:
with open('test.json', 'w') as file:
    json.dump(result, file)

In [ ]:
with io.BytesIO(base64.b64decode(result['b64_strings']['rgb'])) as file:
    im = imageio.imread(file)
viz.imshow(im, figsize=1.5)

In [ ]:
len(result['b64_strings']['rgb'])

### Tile all thumbnails

In [ ]:
query = '''
    select * from microscopy_fov fov
    left join (select * from microscopy_fov_result where kind = '%s') res 
    on fov.id = res.fov_id;'''
df = pd.read_sql(query % 'fov-features', engine)

df['score'] = [data.get('score') for data in df.data]
df.shape

In [ ]:
tns = (
    Session.query(models.MicroscopyThumbnail)
    .filter(models.MicroscopyThumbnail.channel == 'rgb')
    .all()
)
len(tns)

In [ ]:
# test saving thumbnail data
d = [tns[ind].as_dict() for ind in range(1800)]
with open('test.json', 'w') as file: 
    # json.dump(d, file)

In [ ]:
# thumbnails from FOVs with a non-nan score
fov_ids = df.loc[~df.score.isna()].fov_id.values
tns = [tn for tn in tns if tn.fov_id in fov_ids]

In [ ]:
len(tns)

In [ ]:
# sort thumbnails by cell line
tns_sorted = sorted(tns, key=lambda tn: tn.fov.cell_line.id)

In [ ]:
120*192

In [ ]:
size = 102
n_rows = 120
n_cols = 192
tile = np.zeros((n_rows*size, n_cols*size, 3), dtype='uint8')
count = 0
for ind in range(n_rows):
    for jnd in range(n_cols):
        with io.BytesIO(base64.b64decode(tns_sorted[count].data)) as file:
            im = imageio.imread(file)
        tile[ind*size:(ind + 1)*size, jnd*size:(jnd + 1)*size, :] = im
        count += 1

In [ ]:
tile.shape

In [ ]:
imageio.imsave('tile_q90_sorted.jpg', tile[:, :, :], quality=90)

In [ ]:
imageio.imsave('tile_q90_sorted_small.jpg', tile[::6, ::6, :], quality=90)

In [ ]:
viz.imshow(tile[::4, ::4,:], figsize=16)